In [1]:
pip install google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 4.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 5.0 MB/s eta 0:00:0000:01
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 6.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0
  A

In [25]:
from google.cloud import bigquery
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/yangfan0014/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/NUS/Year4_Sem2/IS3107/Project/is3107-news-7c23db21068f.json"

client = bigquery.Client()

dataset_id = 'raw_data'
table_id = 'raw_news_test'


In [16]:
import requests

def get_finance_news(api_key, query, date, language='en', sort = 'publishedAt'):
    """
    Function to fetch finance-related news from News API.

    Args:
    - api_key (str): Your News API key.
    - query (str): The query term to search for (default is 'finance').
    - language (str): The language of the news articles (default is 'en').
    - page_size (int): The number of articles to fetch per request (default is 100, maximum allowed by News API).

    Returns:
    - Pandas DataFrame containing information about news articles.
    """
    url = 'https://newsapi.org/v2/everything?'
    params = {
        'apiKey': api_key,
        'q': query,
        'language': language,
        'sortBy': sort,
        'from': date
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == 'ok':
        for element in data['articles']:
            source_dict = {"source_id": element["source"]["id"],
                           "source_name": element["source"]["name"]}
            element.pop("source")
            element.update(source_dict)
        return data['articles']

In [17]:
api_key = '771d3fbdb9224144a4830d0e33d37c10'
finance_news_df = get_finance_news(api_key, 'fintech', '2024-03-25')

In [23]:
schema = [
    bigquery.SchemaField("author", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("description", "STRING"),
    bigquery.SchemaField("url", "STRING"),
    bigquery.SchemaField("urlToImage", "STRING"),
    bigquery.SchemaField("publishedAt", "STRING"),
    bigquery.SchemaField("content", "STRING"),
    bigquery.SchemaField("source_id", "STRING"),
    bigquery.SchemaField("source_name", "STRING"),
]

In [30]:
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

table = bigquery.Table(table_ref, schema=schema)

table = client.create_table(table)


In [32]:
client.insert_rows_json(table_ref, finance_news_df)

[]